In [1]:
import pandas as pd
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00


In [3]:
from transformers import pipeline

In [4]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_analysis = pipeline("sentiment-analysis", model = model_name, device = 0, truncation = True)
print(sentiment_analysis("I love this !"))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'label': 'positive', 'score': 0.977944016456604}]


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

In [6]:
from google.colab import drive
drive.mount('/gdrive')



Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/qlik/escalation_project/train_9_14

/gdrive/MyDrive/qlik/escalation_project/train_9_14


In [ ]:
%pwd

'/gdrive/MyDrive/qlik/escalation_project/train_9_14'

In [ ]:
train_df = pd.read_csv("escalation_per_comment_train.csv")

In [ ]:
train_df.keys()

Index(['ID', 'PARENT_ID', 'CREATED_AT', 'CREATED_DATE', 'COMMENT_BODY',
       'IS_PUBLISHED', 'CUSTOMER_COMMENT_BOO', 'N_COMMENT',
       'N_CUSTOMER_COMMENT', 'N_SUPPORT_COMMENT', 'HAS_ESCALATED_VAL',
       'N_ESCALATED', 'N_CASE', 'PROP_ESCALATED', 'AVG_PRIORITY',
       'AVG_SEVERITY', 'ACCOUNT_ID', 'N_JIRA_TICKETS_UP_TO_COMMENT',
       'BUG_WEIGHT_MAX_UP_TO_COMMENT', 'BUG_WEIGHT_AVG_UP_TO_COMMENT',
       'SEVERITY_MAX_UP_TO_COMMENT', 'SEVERITY_AVG_UP_TO_COMMENT',
       'SEVERITY_COUNT_UP_TO_COMMENT', 'SEVERITY_LATEST_UP_TO_COMMENT',
       'PRIORITY_MAX_UP_TO_COMMENT', 'PRIORITY_AVG_UP_TO_COMMENT',
       'PRIORITY_COUNT_UP_TO_COMMENT', 'PRIORITY_LATEST_UP_TO_COMMENT'],
      dtype='object')

In [ ]:
from tqdm import tqdm
scores = []

temp = train_df['COMMENT_BODY'].tolist()
for temp_str in tqdm(temp):
  scores.append(sentiment_analysis(temp_str, truncation = True, padding=True, max_length = 512))

  0%|          | 3/151937 [00:00<1:26:12, 29.37it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 151937/151937 [50:59<00:00, 49.66it/s]


In [ ]:
labels =[x[0]['label'] for x in scores]
confidence_scores =[x[0]['score'] for x in scores]

In [ ]:
def get_label_val(s):
  if s == 'negative':
    return 0
  if s == 'neutral':
    return 1
  return 2
label_vals = list(map(get_label_val, labels))

In [ ]:
sentiment_score = np.array(label_vals).astype(np.float) * 1/3 + 1/3 * np.array(confidence_scores).astype(np.float)

<ipython-input-53-4252bd7efd8e>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sentiment_score = np.array(label_vals).astype(np.float) * 1/3 + 1/3 * np.array(confidence_scores).astype(np.float)


In [ ]:
train_df['sentiment_label'] = labels
train_df['sentiment_confidence_score'] = confidence_scores
train_df['sentiment_score'] = sentiment_score

In [ ]:
train_df.to_csv("escalation_per_comment_train_with_sentiment.csv")